In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/NGULIM.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

In [2]:
loan_result = pd.read_csv('loan_result.csv')

In [3]:
data = loan_result.copy()
data['bank_id'] = data['bank_id'].astype('str')
data['product_id'] = data['product_id'].astype('str')
data['bank_product'] = data['bank_id']+' '+data['product_id']
data

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied,bank_product
0,1748340,2022-06-07 13:05:41,7,191,42000000.0,13.6,NaN,7 191
1,1748340,2022-06-07 13:05:41,25,169,24000000.0,17.9,NaN,25 169
2,1748340,2022-06-07 13:05:41,2,7,24000000.0,18.5,NaN,2 7
3,1748340,2022-06-07 13:05:41,4,268,29000000.0,10.8,NaN,4 268
4,1748340,2022-06-07 13:05:41,11,118,5000000.0,16.4,NaN,11 118
...,...,...,...,...,...,...,...,...
13527358,1428218,2022-06-03 12:01:58,62,200,3000000.0,14.8,NaN,62 200
13527359,1428218,2022-06-03 12:01:49,2,7,40000000.0,11.8,NaN,2 7
13527360,1428218,2022-06-03 12:01:48,32,257,15000000.0,7.2,NaN,32 257
13527361,1428218,2022-06-03 12:01:48,33,110,44000000.0,13.5,NaN,33 110


### loan_rate 결측치 처리

+ bank_product 기준으로 평균 보간
+ 채워지지 않는 값은 application_id 기준으로 보간
    + 1.0 -> 가장 낮은 값으로 보간

In [4]:
data['loan_rate'].isnull().sum()

7495

In [5]:
rate = pd.DataFrame(data['loan_rate'])
rate

,loan_rate
0,13.6
1,17.9
2,18.5
3,10.8
4,16.4
...,...
13527358,14.8
13527359,11.8
13527360,7.2
13527361,13.5


In [6]:
ndf = []

for idx, val in data.groupby('bank_product'):
    #print(val)
    #print(idx)
    ndf.append(val['loan_rate'].fillna(val['loan_rate'].mean()))
    
data_fill = pd.concat(ndf,axis=0)

data_fill = data_fill.sort_index()
data['loan_rate'] = data_fill

rate['data_fill'] = data_fill

In [7]:
data['loan_rate'].isnull().sum()

2837

In [8]:
line = []

for idx, val in data.groupby('application_id'):
    #print(val)
    #print(idx)
    line.append(val['loan_rate'].fillna(val['loan_rate'].min()))
                
data_fill_line = pd.concat(line,axis=0)

#data['loan_rate'] = data_fill_line
data_fill_line = data_fill_line.sort_index()

rate['data_fill_line'] = data_fill_line

In [9]:
data['loan_rate'] = data_fill_line

In [10]:
rate

,loan_rate,data_fill,data_fill_line
0,13.6,13.6,13.6
1,17.9,17.9,17.9
2,18.5,18.5,18.5
3,10.8,10.8,10.8
4,16.4,16.4,16.4
...,...,...,...
13527358,14.8,14.8,14.8
13527359,11.8,11.8,11.8
13527360,7.2,7.2,7.2
13527361,13.5,13.5,13.5


In [11]:
rate.isnull().sum()

loan_rate         7495
data_fill         2837
data_fill_line      11
dtype: int64

In [12]:
data.isnull().sum()

application_id                 0
loanapply_insert_time          0
bank_id                        0
product_id                     0
loan_limit                  7495
loan_rate                     11
is_applied               3257239
bank_product                   0
dtype: int64

In [16]:
data

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied,bank_product
0,1748340,2022-06-07 13:05:41,7,191,42000000.0,13.6,NaN,7 191
1,1748340,2022-06-07 13:05:41,25,169,24000000.0,17.9,NaN,25 169
2,1748340,2022-06-07 13:05:41,2,7,24000000.0,18.5,NaN,2 7
3,1748340,2022-06-07 13:05:41,4,268,29000000.0,10.8,NaN,4 268
4,1748340,2022-06-07 13:05:41,11,118,5000000.0,16.4,NaN,11 118
...,...,...,...,...,...,...,...,...
13527358,1428218,2022-06-03 12:01:58,62,200,3000000.0,14.8,NaN,62 200
13527359,1428218,2022-06-03 12:01:49,2,7,40000000.0,11.8,NaN,2 7
13527360,1428218,2022-06-03 12:01:48,32,257,15000000.0,7.2,NaN,32 257
13527361,1428218,2022-06-03 12:01:48,33,110,44000000.0,13.5,NaN,33 110


data.to_csv('loan_rate_fill.csv',index=False)

### loan_limit 전처리

+ desired_amount와 연관이 높다
    + desired_amount로 결측값 채우기
    + 채우기 전에 is_applied = 1.0인 것 확인

data = pd.read_csv('loan_rate_fill.csv')

In [17]:
user_spec_fillna = pd.read_csv('user_spec_fillna.csv')
user_spec_fillna

,application_id,user_id,birth_year,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,ymd
0,1031304,1,2000.0,1.0,2022-05-03 14:51:00,750.0,20000000.0,OTHERINCOME,NaN,기타,전월세,0.0,생활비,0.0,-1.0,1.0,10000000.0,2022-05-03
1,1767402,1,2000.0,1.0,2022-05-03 14:53:33,750.0,20000000.0,OTHERINCOME,NaN,기타,전월세,1000000.0,생활비,0.0,-1.0,1.0,10000000.0,2022-05-03
2,1096882,7,1985.0,1.0,2022-05-22 16:42:47,580.0,22000000.0,EARNEDINCOME2,201907.0,계약직,기타가족소유,10000000.0,생활비,0.0,-1.0,7.0,46000000.0,2022-05-22
3,248576,9,1996.0,1.0,2022-05-21 23:41:09,810.0,29000000.0,EARNEDINCOME,202201.0,계약직,기타가족소유,15000000.0,사업자금,0.0,-1.0,1.0,10000000.0,2022-05-21
4,1180616,11,1997.0,1.0,2022-03-24 10:58:22,640.0,25000000.0,EARNEDINCOME,202106.0,정규직,기타가족소유,2000000.0,생활비,0.0,-1.0,1.0,14000000.0,2022-03-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1394190,1889786,879694,1991.0,0.0,2022-03-31 20:17:31,NaN,39000000.0,EARNEDINCOME,201909.0,정규직,전월세,10000000.0,생활비,0.0,-1.0,0.0,0.0,2022-03-31
1394191,1774704,879695,1996.0,1.0,2022-05-27 12:50:55,NaN,30000000.0,EARNEDINCOME2,202202.0,정규직,전월세,15000000.0,대환대출,0.0,-1.0,0.0,0.0,2022-05-27
1394192,2149910,879696,1997.0,1.0,2022-03-14 05:23:30,610.0,37000000.0,EARNEDINCOME,202110.0,정규직,전월세,10000000.0,생활비,0.0,-1.0,3.0,20000000.0,2022-03-14
1394193,1856758,879696,1997.0,1.0,2022-03-14 05:40:38,610.0,37000000.0,EARNEDINCOME,202110.0,정규직,전월세,10000000.0,생활비,0.0,-1.0,3.0,20000000.0,2022-03-14


In [18]:
user = user_spec_fillna[['application_id','desired_amount']]
user = pd.merge(data,user,how='left',on='application_id')
user

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied,bank_product,desired_amount
0,1748340,2022-06-07 13:05:41,7,191,42000000.0,13.6,NaN,7 191,25000000.0
1,1748340,2022-06-07 13:05:41,25,169,24000000.0,17.9,NaN,25 169,25000000.0
2,1748340,2022-06-07 13:05:41,2,7,24000000.0,18.5,NaN,2 7,25000000.0
3,1748340,2022-06-07 13:05:41,4,268,29000000.0,10.8,NaN,4 268,25000000.0
4,1748340,2022-06-07 13:05:41,11,118,5000000.0,16.4,NaN,11 118,25000000.0
...,...,...,...,...,...,...,...,...,...
13527358,1428218,2022-06-03 12:01:58,62,200,3000000.0,14.8,NaN,62 200,50000000.0
13527359,1428218,2022-06-03 12:01:49,2,7,40000000.0,11.8,NaN,2 7,50000000.0
13527360,1428218,2022-06-03 12:01:48,32,257,15000000.0,7.2,NaN,32 257,50000000.0
13527361,1428218,2022-06-03 12:01:48,33,110,44000000.0,13.5,NaN,33 110,50000000.0


In [19]:
line = []

for idx, val in user.groupby('application_id'):
    #print(val)
    #print(idx)
    line.append(val['loan_limit'].fillna(val['desired_amount']))
                
user_spec_fill_line = pd.concat(line,axis=0)

In [20]:
user_spec_fill_line = user_spec_fill_line.sort_index()
print(user_spec_fill_line.isnull().sum())
user_spec_fill_line

113


0           42000000.0
1           24000000.0
2           24000000.0
3           29000000.0
4            5000000.0
               ...    
13527358     3000000.0
13527359    40000000.0
13527360    15000000.0
13527361    44000000.0
13527362    44000000.0
Name: loan_limit, Length: 13527363, dtype: float64

In [21]:
data['loan_limit'] = user_spec_fill_line
data.isnull().sum()

application_id                 0
loanapply_insert_time          0
bank_id                        0
product_id                     0
loan_limit                   113
loan_rate                     11
is_applied               3257239
bank_product                   0
dtype: int64

In [22]:
data

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied,bank_product
0,1748340,2022-06-07 13:05:41,7,191,42000000.0,13.6,NaN,7 191
1,1748340,2022-06-07 13:05:41,25,169,24000000.0,17.9,NaN,25 169
2,1748340,2022-06-07 13:05:41,2,7,24000000.0,18.5,NaN,2 7
3,1748340,2022-06-07 13:05:41,4,268,29000000.0,10.8,NaN,4 268
4,1748340,2022-06-07 13:05:41,11,118,5000000.0,16.4,NaN,11 118
...,...,...,...,...,...,...,...,...
13527358,1428218,2022-06-03 12:01:58,62,200,3000000.0,14.8,NaN,62 200
13527359,1428218,2022-06-03 12:01:49,2,7,40000000.0,11.8,NaN,2 7
13527360,1428218,2022-06-03 12:01:48,32,257,15000000.0,7.2,NaN,32 257
13527361,1428218,2022-06-03 12:01:48,33,110,44000000.0,13.5,NaN,33 110


In [23]:
data.to_csv('loan_result_fillna.csv')

data.dropna(axis=0, subset=['loan_rate','loan_limit'])

loan_rate_fill = data['loan_rate'].dropna(axis=0)

loan_rate_fill

'''
user.groupby(['bank_product','loan_rate']).mean()#['loan_limit']

amt_list = []

for idx, val in user.groupby(['bank_product','loan_rate']):
    #print(val)
    #print(idx)
    amt_list.append(val['loan_limit'].fillna(val['loan_limit'].mean()))
                
user_amt = pd.concat(amt_list,axis=0)
user_amt

amt = pd.DataFrame(user['loan_limit'])
amt

user_amt = user_amt.sort_index()
amt['loan_limit_fill'] = user_amt
amt

amt.isnull().sum()
'''

a = loan_rate_fill[loan_rate_fill['loan_limit'].isnull()]

for i in a['loanapply_insert_time']:
    print(i)

+ 결측값인 행이 테스트에 없음 -> drop 해도 괜찮다